In [10]:
import numpy as np
import pandas as pd

In [11]:
!pip install pyquaternion

In [12]:
#http://kieranwynn.github.io/pyquaternion
from pyquaternion import Quaternion

def to_quaternion(list_of_vectors):
  new_list = []
  for one_vector in list_of_vectors:
    new_list.append(Quaternion(one_vector))
  return new_list

def first_moment(list_of_vectors):
  q = np.sum(list_of_vectors, axis=0) / len(list_of_vectors)
  return Quaternion(q)

def second_moment(list_of_vectors):
  tmp_vectors = []
  for vector in list_of_vectors:
    Q = Quaternion(vector)
    Q_conj = Q.conjugate
    tmp_vectors.append(Q*Q_conj)
  return first_moment(tmp_vectors)

def third_moment(list_of_vectors):
  tmp_vectors = []
  for vector in list_of_vectors:
    Q = Quaternion(vector)
    Q_conj = Q.conjugate
    tmp_vectors.append(Q*Q_conj*Q)
  return first_moment(tmp_vectors)

def compute_moments(list_of_vectors):
  list_of_vectors = to_quaternion(list_of_vectors)
  m1 = first_moment(list_of_vectors)
  m2 = second_moment(list_of_vectors)
  m3 = third_moment(list_of_vectors)
  return [m1,m2,m3]

In [5]:
import math
def calculate_z(M1,M2,M3):
  C1 = (M3 - M1*M2)/(M1*M1.conjugate - M2)
  C0 = -(M1.conjugate*C1 + M2)
  # Extration of component
  c10, c11, c12, c13 = C1
  c00,   _,   _,   _ = C0
  # Computing temporal variables
  pow_c13c12 = c13*c13 + c12*c12
  U = (c10*c12 - c11*c13)/pow_c13c12
  V = (c11*c12 + c10*c13)/pow_c13c12
  W = (c11*V + c10*U + c12)/(c10*V - c11*U + c13)
  a1 = ( 1 + W*W )*( 1 + U*U + V*V)
  a2 = W * ( U*c10 + V*c11 + c12) - U*c11 + V*c10 + c13
  # Solving a1^2 * z3^2 + a2*z3 + c00 = 0
  d = math.sqrt( a2*a2 - 4*a1*c00 ) 
  z03 = - (a2 - d) / (2*a1)
  z13 = - (a2 + d) / (2*a1)
  z02 = W * z03
  z12 = W * z13
  z01 = (W*V - U) * z03
  z11 = (W*V - U) * z13
  z00 = (W*U + V) * z03
  z10 = (W*U + V) * z13
  Z0 = Quaternion(z00, z01, z02, z03)
  Z1 = Quaternion(z10, z11, z12, z13)
  return Z1, Z0

In [6]:
def decision_boundary(Z0,Z1):
  z10, z11, z12, z13 = Z1
  z00, z01, z02, z03 = Z0
  Z0Z0 = z00*z00 + z01*z01 + z02*z02 + z03*z03
  Z1Z1 = z10*z10 + z11*z11 + z12*z12 + z13*z13
  s = (Z0Z0-Z1Z1)/(2*(z00-z10))
  t1 = (z01 - z11)/(z00-z10)
  t2 = (z02 - z12)/(z00-z10)
  t3 = (z03 - z13)/(z00-z10)
  return s,t1,t2,t3

def clasification(Q,Z0,Z1):
  s, t1, t2, t3 = decision_boundary(Z0,Z1)
  bitmap = []
  count_1 = 0
  count_0 = 0
  for one_quaternio in Q:
    q0, q1, q2, q3 = one_quaternio
    q_diff = q0 - (s - t1*q1 - t2*q2 - t3*q3)
    if q_diff > 0:
      bitmap.append(1)
      count_1 = count_1 + 1
    else:
      bitmap.append(0)
      count_0 = count_0 + 1
  p0 = count_0 / (count_0+count_1)
  p1 = count_1 / (count_0+count_1)
  return np.array(bitmap) , p0 , p1

In [27]:
def BQMP_thresholding(Q_data):
  Q = to_quaternion(Q_data)
  M1, M2, M3 = compute_moments(Q)
  # Doing M1 = 0, by preserve moments
  Q_ = Q - np.array( M1 )
  M1_, M2_, M3_ = compute_moments(Q_)
  #print("M1  : {0} \n M2  : {1} \n M3  : {0}".format(M1, M2, M3) )
  #print("M1_ : {0} \n M2_ : {1} \n M3_ : {0}".format(M1_, M2_, M3_) )
  # Calculate Z0 & Z1
  Z1_, Z0_ = calculate_z(M1_,M2_,M3_)
  # Real value Z0 & Z1
  Z1 = Z1_ + M1
  Z0 = Z0_ + M1
  # Approximate to p0 & p1
  array_bitmap, p0, p1 = clasification(Q,Z0,Z1)
  return Z1, Z0, array_bitmap, p0, p1

In [31]:
Q =[[  8, 12, 19,  9], [  7, 13, 14, 13], [ 11, 14, 19, 10], [ 13, 10, 16,  9],
    [ 80,117, 53, 98], [ 88,118, 63,103], [105,106, 88, 89], [109,117, 71,106],
    [ 90,112, 49, 92], [ 99,113, 34, 84], [ 10, 17, 17, 18], [113,115, 77,117],
    [100,117, 34, 89], [102,115, 43, 93], [ 12, 13, 12, 17], [110,112, 70,122]]
Q = np.array(Q)
print("Q      : \n {0} \n".format(Q))
Z1, Z0, bitmap, p0, p1 = BQMP_thresholding(Q)
print("Z0     : ", Z0)
print("Z1     : ", Z1)
print("p0     : ", p0)
print("p1     : ", p1)
print("bitmap : ", bitmap)

Q      : 
 [[  8  12  19   9]
 [  7  13  14  13]
 [ 11  14  19  10]
 [ 13  10  16   9]
 [ 80 117  53  98]
 [ 88 118  63 103]
 [105 106  88  89]
 [109 117  71 106]
 [ 90 112  49  92]
 [ 99 113  34  84]
 [ 10  17  17  18]
 [113 115  77 117]
 [100 117  34  89]
 [102 115  43  93]
 [ 12  13  12  17]
 [110 112  70 122]] 

Z0     :  101.780 +119.593i +56.310j +100.844k
Z1     :  12.050 +10.864i +21.460j +15.351k
p0     :  0.375
p1     :  0.625
bitmap :  [0 0 0 0 1 1 1 1 1 1 0 1 1 1 0 1]
